## Bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import seaborn
from matplotlib import pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

## Carregando Treino

In [2]:
train = pd.read_csv('santander-customer-transaction-prediction/train.csv')
train.dtypes

ID_code     object
target       int64
var_0      float64
var_1      float64
var_2      float64
            ...   
var_195    float64
var_196    float64
var_197    float64
var_198    float64
var_199    float64
Length: 202, dtype: object

In [3]:
train.describe()

target          var_0          var_1          var_2  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean        0.100490      10.679914      -1.627622      10.715192   
std         0.300653       3.040051       4.050044       2.640894   
min         0.000000       0.408400     -15.043400       2.117100   
25%         0.000000       8.453850      -4.740025       8.722475   
50%         0.000000      10.524750      -1.608050      10.580000   
75%         0.000000      12.758200       1.358625      12.516700   
max         1.000000      20.315000      10.376800      19.353000   

               var_3          var_4          var_5          var_6  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean        6.796529      11.078333      -5.065317       5.408949   
std         2.043319       1.623150       7.863267       0.866607   
min        -0.040200       5.074800     -32.562600       2.347300   
25%         5.254075       9.883175     -11.200350       4.767700   
50%         6.825000      11.108250      -4.833150       5.385100   
75%         8.324100      12.261125       0.924800       6.003000   
max        13.188300      16.671400      17.251600       8.447700   

               var_7          var_8          var_9         var_10  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean       16.545850       0.284162       7.567236       0.394340   
std         3.418076       3.332634       1.235070       5.500793   
min         5.349700     -10.505500       3.970500     -20.731300   
25%        13.943800      -2.317800       6.618800      -3.594950   
50%        16.456800       0.393700       7.629600       0.487300   
75%        19.102900       2.937900       8.584425       4.382925   
max        27.691800      10.151300      11.150600      18.670200   

              var_11         var_12         var_13         var_14  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean       -3.245596      14.023978       8.530232       7.537606   
std         5.970253       0.190059       4.639536       2.247908   
min       -26.095000      13.434600      -6.011100       1.013300   
25%        -7.510600      13.894000       5.072800       5.781875   
50%        -3.286950      14.025500       8.604250       7.520300   
75%         0.852825      14.164200      12.274775       9.270425   
max        17.188700      14.654500      22.331500      14.937700   

              var_15         var_16         var_17         var_18  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean       14.573126       9.333264      -5.696731      15.244013   
std         0.411711       2.557421       6.712612       7.851370   
min        13.076900       0.635100     -33.380200     -10.664200   
25%        14.262800       7.452275     -10.476225       9.177950   
50%        14.574100       9.232050      -5.666350      15.196250   
75%        14.874500      11.055900      -0.810775      21.013325   
max        15.863300      17.950600      19.025900      41.748000   

              var_19         var_20         var_21         var_22  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean       12.438567      13.290894      17.257883       4.305430   
std         7.996694       5.876254       8.196564       2.847958   
min       -12.402500      -5.432200     -10.089000      -5.322500   
25%         6.276475       8.627800      11.551000       2.182400   
50%        12.453900      13.196800      17.234250       4.275150   
75%        18.433300      17.879400      23.089050       6.293200   
max        35.183000      31.285900      49.044300      14.594500   

              var_23         var_24         var_25         var_26  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean        3.019540      10.584400      13.667496      -4.055133   
std         0.526893       3.777245       0.285535       5.922210   
min   

In [4]:
train.isna().sum().sum()

np.int64(0)

In [5]:
X_train, y_train = train.iloc[:, 2:], train['target']
X_train

var_0   var_1    var_2   var_3    var_4    var_5   var_6    var_7  \
0        8.9255 -6.7863  11.9081  5.0930  11.4607  -9.2834  5.1187  18.6266   
1       11.5006 -4.1473  13.8588  5.3890  12.3622   7.0433  5.6208  16.5338   
2        8.6093 -2.7457  12.0805  7.8928  10.5825  -9.0837  6.9427  14.6155   
3       11.0604 -2.1518   8.9522  7.1957  12.5846  -1.8361  5.8428  14.9250   
4        9.8369 -1.4834  12.8746  6.6375  12.2772   2.4486  5.9405  19.2514   
...         ...     ...      ...     ...      ...      ...     ...      ...   
199995  11.4880 -0.4956   8.2622  3.5142  10.3404  11.6081  5.6709  15.1516   
199996   4.9149 -2.4484  16.7052  6.6345   8.3096 -10.5628  5.8802  21.5940   
199997  11.2232 -5.0518  10.5127  5.6456   9.3410  -5.4086  4.5555  21.5571   
199998   9.7148 -8.6098  13.6104  5.7930  12.5173   0.5339  6.0479  17.0152   
199999  10.8762 -5.7105  12.1183  8.0328  11.5577   0.3488  5.2839  15.2058   

         var_8   var_9  var_10   var_11   var_12   var_13   var_14   var_15  \
0      -4.9200  5.7470  2.9252   3.1821  14.0137   0.5745   8.7989  14.5691   
1       3.1468  8.0851 -0.4032   8.0585  14.0239   8.4135   5.4345  13.7003   
2      -4.9193  5.9525 -0.3249 -11.2648  14.1929   7.3124   7.5244  14.6472   
3      -5.8609  8.2450  2.3061   2.8102  13.8463  11.9704   6.4569  14.8372   
4       6.2654  7.6784 -9.4458 -12.1419  13.8481   7.8895   7.7894  15.0553   
...        ...     ...     ...      ...      ...      ...      ...      ...   
199995 -0.6209  5.6669  3.7574  -9.5348  13.9860   5.2982   8.2705  14.1527   
199996 -3.6797  6.0019  6.5576 -11.8776  14.4131   3.3087   3.5800  14.1597   
199997  0.1202  6.1629  4.4004  -0.4651  13.8775   9.7414  10.9044  14.5597   
199998 -2.1926  8.7542  1.4245   0.7086  14.2110   6.5641   7.6177  13.8771   
199999 -0.4541  9.3688 -7.3826  -8.7049  14.2486  15.0849   5.2313  14.3572   

         var_16   var_17   var_18   var_19   var_20   var_21   var_22  var_23  \
0        5.7487  -7.2393   4.2840  30.7133  10.5350  16.2191   2.5791  2.4716   
1       13.8275 -15.5849   7.8000  28.5708   3.4287   2.7407   8.5524  3.3716   
2        7.6782  -1.7395   4.7011  20.4775  17.7559  18.1377   1.2145  3.5137   
3       10.7430  -0.4299  15.9426  13.7257  20.3010  12.5579   6.8202  2.7229   
4        8.4871  -3.0680   6.5263  11.3152  21.4246  18.9608  10.1102  2.7142   
...         ...      ...      ...      ...      ...      ...      ...     ...   
199995   7.4540  -5.0105  12.0465   8.6349   9.9137  25.1376   1.0914  3.2326   
199996   7.5191  -8.8715  17.9467  17.0237   6.6459  18.2345   0.8982  2.2532   
199997   9.6214  -1.6429  23.1127  12.1517  16.2577   3.1453   3.1008  2.1497   
199998   9.0479 -11.8164  14.0831  -2.0345  18.3863   3.0911   5.5803  3.7091   
199999  12.5523  -6.5066  11.3592  11.4779  15.4997   3.8474   2.4381  2.8295   

         var_24   var_25   var_26  var_27  var_28  var_29   var_30   var_31  \
0       14.3831  13.4325  -5.1488 -0.4073  4.9306  5.9965  -0.3085  12.9041   
1        6.9779  13.8910 -11.7684 -2.5586  5.0464  0.5481  -9.2987   7.8755   
2        5.6777  13.2177  -7.9940 -2.9029  5.8463  6.1439 -11.1025  12.4858   
3       12.1354  13.7367   0.8135 -0.9059  5.9070  2.8407 -15.2398  10.4407   
4       14.2080  13.5433   3.1736 -3.3423  5.9015  7.9352  -3.1582   9.4668   
...         ...      ...      ...     ...     ...     ...      ...      ...   
199995   7.7802  13.9939   2.9085  0.1005  4.2369  7.5665  -9.2149   9.5746   
199996  15.4977  13.3282   5.2281 -3.7424  5.5144  5.7148 -13.7470   7.4369   
199997  10.2715  13.5637   4.9473 -0.9905  6.2801  9.4902 -12.8549  11.0403   
199998  12.8219  13.8866  -3.3859 -0.4440  5.4817  4.0902  -7.7085  10.3952   
199999  10.6681  13.7167  -7.7771 -2.7798  6.2885  6.0089   2.1547  10.8181   

        var_32   var_33   var_34   var_35  var_36  var_37   var_38  var_39  \
0      -3.8766  16.8911  11.1920  10.5785  0.6764  7.8871   4.6667  3.8743   
1       1.2859  19.3710  11.3702  

## Otimização de Modelo e Hiperparâmetro e Treinamento do Modelo

In [6]:
pipelines = [
    {
        'name': 'Stochastic Gradient Descent Classifier',
        'classifier_class': SGDClassifier,
        'param_grid': {
            'classifier__penalty': ['l2', 'elasticnet'],
            'classifier__alpha': [0.0001, 0.001]
        }
    },
    {
        'name': 'Multi-layer Perceptron Classifier',
        'classifier_class': MLPClassifier,
        'param_grid': {
            'classifier__hidden_layer_sizes': [(50,), (100,)],
            'classifier__activation': ['relu', 'tanh'],
            'classifier__max_iter': [1000]
        }
    },
    {
        'name': 'Random Forest Classifier',
        'classifier_class': RandomForestClassifier,
        'param_grid': {
            'classifier__n_estimators': [50, 100],
            'classifier_max_depth': [5, 10]
        }
    },
    {
        'name': 'XGBoost Classifier',
        'classifier_class': XGBClassifier,
        'param_grid': {
            'classifier__n_estimators': [50, 100],
            'classifier__max_depth': [3, 5]
        }
    }
]

In [7]:
n_splits = 5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

best_model = None
best_score = -1
best_params = {}
best_model_name = ""

In [8]:
for model_config in pipelines:
    model_name = model_config['name']
    classifier_class = model_config['classifier_class']
    param_grid = model_config['param_grid']

    classifier = classifier_class(random_state=42)        
 
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', classifier)
    ])

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=skf,
        scoring='accuracy',
        n_jobs=4,
        verbose=1
    )
    grid_search.fit(X_train, y_train)

print(f"Best Model: {best_model_name}")
print(f"Best Accuracy: {best_score:.4f}")
print(f"Best Parameters: {best_params}")
print(f"Best Estimator: {best_model}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/home/rene/miniconda3/envs/test/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/rene/miniconda3/envs/test/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/rene/miniconda3/envs/test/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/rene/miniconda3/envs/test/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

## Avaliação em Teste

## Salvando Modelo